# Présentation de Pandas

Cette présentation est destinée à donner un aperçu de Pandas utilisé pour pour une exploration de données. Il fait suite au document présentant les `Series` et les `DataFrame`.

Vous pouvez consuler [la documentation de Pandas](https://pandas.pydata.org/pandas-docs/stable/) en ligne.

Nous commençons par construire la référence vers le fichier à utiliser et charger Pandas.

In [ ]:
from pathlib import Path
file_path = Path('__file__').resolve().parent.parent / 'assets' / 'comptage-voyageurs-trains-transilien-old.csv'

In [ ]:
import pandas as pd

## Données à partir d'un fichier
Pandas propose une méthode `.read_csv()` pour lire les données dans un csv. Pandas propose également d'autres méthodes pour d'autres sources tel que `.read_json()` ou `.read_sql_table()`.

In [ ]:
train_data = pd.read_csv(file_path, sep=";")

## Structure d'une données Pandas
Nous pouvons vérifier que le type de donnée récupéré par Pandas.

In [ ]:
type(train_data)

Il s'agit bien d'un `DataFrame`.

Nous allons pouvoir consulter son contenu. Dans un notebook, le DataFrame est affiché sous forme de tableau. Mais dans la pratique, nous utiliserons les méthodes `.head()` ou `.tail()` qui nous retournent les 5 premières ou dernières données. Ceci facilite une lecture limitée à un aperçu des données.

In [ ]:
train_data.head()

Nous avons vu qu'un `DataFrame` est une donnée structurée qui possède des **colonnes**, des **indices** et des **données**. Nous allons les consulter.

In [ ]:
train_data.columns

In [ ]:
train_data.index

In [ ]:
train_data.values

Et enfin les dimensions d'un DataFrame qui sont des **axes**

In [ ]:
train_data.axes

## Explorons nos données

Une variable contenant un DataFrame rertourne celui-ci et Jupyter formatera l'affichage. Nous avons vu que nous n'avons pas à récupérer toutes les données, les méthodes `.head()` et `.tail()` nous permettent de ne voir qu'un extrait.

Les colonnes d'un DataFrame contiennent des valeurs d'un type spécifique. Leur parcours est donc bien plus performant qu'une liste Python.

Nous pouvons afficher les colonnes et leur type avec le retour de la méthode `.info()`.

In [ ]:
train_data.info()

## Statistiques de base
Nous pouvons obtenir des statistiques de base grâce à la méthode `.describe()`. Celle-ci décrit toutes les colonnes *numériques*. Nous obtenons le nombre de données, la moyenne globale, la déviation standard, le minimum, les quartiles et le maximum de la Serie

Cette méthode possède un paramètre optionnel `include` qui permet de demander à décrire les autres données.

In [ ]:
#train_data.describe()
#train_data.describe(include="object")
train_data.describe(include="all")

Un DataFrame peut être manipulé comme un dictionnaire pour lequel les clefs seront les colonnes.

In [ ]:
train_data.keys()

In [ ]:
montants = train_data['Montants']
montants

Il ne s'agit pas uniquement d'une *liste* de données. Elles sont indéxées. Un `DataFrame` est composés d'objets de type `Series` qui est comparable au type `List` ou plutôt les `ndarray` Numpy.

In [ ]:
type(montants)

Les `Series` ont donc un ensemble de méthodes comparables aux `ndarray`. Nous avons donc à disposition les méthodes des `ndarray`.

In [ ]:
train_data['Montants'].max()

Et certaines plus spécifiques. La méthode `.value_count()` par exemple retourne un nouvel objet de type `Series`.

In [ ]:
train_data['Ligne'].value_counts()

Notez que vous pouvez souhaiter obtenir une donnée de type `DataFrame` et non `Series` lors de cette opération. Il faut alors accéder à une donnée de type `list`. En d'autres termes, au lieu de passer une chaine de caractères représentant la colonne, il faut passer une liste contenant cette chaine de caractères.

In [ ]:
train_data[['Montants']]

Et bien évidemment, cette syntaxe permet de sélectionner plusieurs colonnes et retourne donc systématiquement un `dataframe`.

In [ ]:
train_data[['Nom gare', 'Montants']]

## Filtrer les données (requêtes)
Filtrer les colonnes est une chose, mais nous souhaitons également filtrer les données (lignes). Ceci correspond à faire des requêtes. Pandas permet de filtrer les données selon un ou plusieurs critères. Nous allons utiliser les _filtres_.

In [ ]:
train_data['Nom gare'] == "PARIS NORD"

In [ ]:
paris_nord_data = train_data.loc[(train_data['Nom gare'] == "PARIS NORD")]
paris_nord_data.head()

La méthode `.loc()` permet de sélectionner en même temps les colonnes de la donnée retournée. Pour cela, nous fournissont celles-ci sous forme d'un second paramètre.

Mais attention, le comprtement est le même que l'approche _dictionnaire_ c'est à dire que si une chaine de caractère est passée (sélection d'une seule colonne), il sera retourné une `Series`. Pour récupérer un DataFrame, il faudra passer une liste.

In [ ]:
train_data.loc[(train_data['Nom gare'] == "PARIS NORD", "Montants")].head()

In [ ]:
train_data.loc[(train_data['Nom gare'] == "PARIS NORD", ["Montants"])].head()

Nous pouvons combiner plusieurs critères séparés avec les opérateurs `|` et `&`. Attention, il est indispensable d'ajouter les parenthèqes pour la priorité.

In [ ]:
train_data.loc[(train_data['Nom gare'] == "PARIS NORD") 
                & (train_data['Ligne'] == "B")]

Nous pouvons évidemment combiner un ensemble de requêtes. La ligne suivante permet d'isoler les données correspondant au nombre de montants max.

In [ ]:
train_data.loc[
    train_data['Montants'] == train_data['Montants'].max()
    ]

## (Ré)Organiser les données
Les données d'un DataFrame sont ordonnées selon le contenu de la donnée d'entrée. Ici le CSV. Nous pouvons les ré-ordonner selon une ou plusieurs colonnes. Ceci nous retourne un nouveau DataFrame.

### Tri selon le nom des gares
Ci-dessous, nous retrouvons un dataframe trié par le nom des gares.

In [ ]:
train_data.sort_values(by=["Nom gare"])

### Tri selon plusieurs critères
Et bien sûr, nous pouvons passer plusieurs colonnes et donc trier selon plusieurs critères.

In [ ]:
train_data.sort_values(by=["Date de comptage", "Nom gare"])

### Tri par ordre chronologique des dates
Une réorganisation intéressante ici serait une réorganisation chronologique.

Pour une question de quantité de données (et de cohérence des données étudiées), nous allons filtrer les données et ne travailler que sur les comptages de **la ligne B** à **Paris Nord**.

In [ ]:
paris_nord_b = train_data.loc[
        (train_data['Nom gare'] == "PARIS NORD") & (train_data['Ligne'] == "B")
    ] [["Date de comptage", "Tranche horaire", "Type jour", "Montants"]]

Nous pouvons donc réorganiser les données par date et tranche horaire. Sauf que cette dernière donnée est une chaine de caractères et que l'ordre sera donc celui du tri de chaines de caractères.

In [ ]:
paris_nord_b.sort_values(by=["Date de comptage", "Tranche horaire"]).head(10)

Pour avoir une action cohérente, la méthode `.sort_values()` contient un paramètre `key` comme la méthode `.sort()` des listes (ou la fonction `sorted()`) permettant de préciser le critère de tri. Ce que nous allons faire, c'est utiliser une lambda et un dictionnaire. Ce dernier donne un ordre à chaque chaine, la lambda permet de récupérer la donnée du dictionnaire.

Le dictionnaire est le suivant :

In [ ]:
horaires = {
    "Avant 6h": 0,
    "De 6h à 10h": 1,
    "De 10h à 16h": 2,
    "De 16h à 20h": 3,
    "Après 20h": 4,
}

Cependant, pour le paramètre `key`, il y a une différence avec le `.sort()` classique : celui-ci ne prends pas une valeur mais une `Series`. Nous devrons donc appliquer la méthode `.map()` des Series qui transforme la Series ([voir doc de Series.map()](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html))

La lambda serait donc la suivante :
```python
lambda val: val.map(horaires)
```

À titre d'information, le résultat sera le suivant :

In [ ]:
paris_nord_b["Tranche horaire"].map(horaires)

Cependnant, si nous écrivons :
```python
paris_nord_b.sort_values(by=["Date de comptage", "Tranche horaire"],
                         key=lambda val: val.map(horaires)).head(10)
```

Nous aurons une erreur car la méthode `.sort_values()` appliquera la méthode `key` à **toutes** les colonnes de tri…

Il y aura donc une erreur car il ne pourra pas faire de map sur `"Date de comptage"`.

Il ne faut donc appliquer le map que sur la colonne concernée. Cette "vérification" sera appliquée dans la lambda.

In [ ]:
paris_nord_b_sorted = paris_nord_b.sort_values(by=["Date de comptage", "Tranche horaire"],
                                               key=lambda val: val
                                                   if val.name != "Tranche horaire"
                                                   else val.map(horaires))

In [ ]:
paris_nord_b_sorted.head(10)

## Grouper les données
La méthode `groupby` permet, de créer des groupes de données.

Il ne s'agit pas d'une réorganisation des données. Le type obtenu n'est pas un `DataFrame` mais un `DataFrameGroupBy`.

In [ ]:
paris_nord_b_group = paris_nord_b_sorted.groupby('Date de comptage')
type(paris_nord_b_group)

Ce type est un itérable retournant un N-uplet de 2 valeurs :

In [ ]:
for name, group in paris_nord_b_group:
    print(f'Nom : "{name}"')
    print(group)
    print(type(group))

Pandas fournit des opération d'aggrégation, c'est à dire qu'elles permettent une opération sur l'ensemble du groupe.

In [ ]:
paris_nord_b_group["Montants"].sum()

In [ ]:
paris_nord_b_group["Montants"].mean()

In [ ]:
paris_nord_b_group["Montants"].std()

Il est possible de créer des groupes sur plusieurs colonnes, le nom sera alors la liste des colonnes.

L'exemple suivant filtre les données pour la gare de Paris Nord puis les trie par ordre chronologique. Ce dataframe est ensuite groupé par ligne et date.

In [ ]:
paris_nord = train_data.loc[train_data['Nom gare'] == "PARIS NORD"] \
                         [["Ligne", "Date de comptage", "Tranche horaire", "Type jour", "Montants"]]

paris_nord_sorted = paris_nord.sort_values(by=["Date de comptage", "Tranche horaire"],
                                           key=lambda val: val if val.name != "Tranche horaire" else val.map(horaires))

paris_nord_groups = paris_nord_sorted.groupby(["Ligne", "Date de comptage"])

groupe_iterator = paris_nord_groups.__iter__()
print(groupe_iterator)
print()

name, group = next(groupe_iterator)
print(f'Nom : "{name}"')
print(group)
print(type(group))

Un `DataFrameGroupBy` est une collection. Il est possible d'accéder à la donnée (au DataFrame) d'un groupe par le nom du groupe :

In [ ]:
paris_nord_b_group.get_group('2014-03-27')

In [ ]:
paris_nord_groups.get_group(('K', '2018-11-15'))

## Graphes simples avec Pandas
Plus d'informations dans [la documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html)

L'exemple suivant montre comment générer des graphes simples avec la méthode `.plot()` de Pandas.

Nous allons travailler sur la gare de Paris Saint Lazare.

In [ ]:
psl_values = train_data.loc[
    (train_data["Nom gare"] == "PARIS SAINT-LAZARE") & (train_data['Type jour'] == "JOB")
    ].sort_values(by=["Date de comptage", "Tranche horaire"],
                  key=lambda val: val 
                  if val.name != "Tranche horaire" 
                  else val.map(horaires)) [["Nom gare", "Date de comptage", "Tranche horaire", "Ligne", "Montants"]]

In [ ]:
psl_values

Sur un `DataFrameGroupBy`, nous aurons un graph par groupe.

In [ ]:
psl_values.groupby('Date de comptage').plot(x="Tranche horaire", kind="bar")

In [ ]:
for title, group in psl_values.groupby('Date de comptage'):
    group.plot(x="Tranche horaire", kind="bar", title=title)